In [ ]:
import lightgbm as lgb
from sklearn.metrics import mean_pinball_loss

# y: daily (or cumulative) target
# X: engineered features
train_mask = (X["date"] < "YYYY-MM-DD")  # pick a cutoff
X_tr, y_tr = X[train_mask].drop(columns=["date","rm_id"]), y[train_mask]
X_va, y_va = X[~train_mask].drop(columns=["date","rm_id"]), y[~train_mask]

model = lgb.LGBMRegressor(
    objective="quantile",
    alpha=0.2,
    n_estimators=1000,
    learning_rate=0.05,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42
)
model.fit(X_tr, y_tr,
          eval_set=[(X_va, y_va)],
          eval_metric=lambda y_true,y_pred: ("p20", mean_pinball_loss(y_true, y_pred, alpha=0.2), False),
          verbose=50)

va_pred = model.predict(X_va)
